# Service React Agent

All code is written based on:
1. Artcile: [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629)
2. Post: [Building a Python React Agent Class: A Step-by-Step Guide](https://www.neradot.com/post/building-a-python-react-agent-class-a-step-by-step-guide)

This code is more acurrate and separate action understanding driver from the action execution driver. Formaly it can be splited to the different services.

## Imports

In [1]:
import asyncio
import json
import traceback
import uuid
from dataclasses import dataclass

from openai import AsyncOpenAI
from sqlalchemy import JSON, Boolean, Column, ForeignKey, Integer, String, select
from sqlalchemy.ext.asyncio import AsyncSession, create_async_engine
from sqlalchemy.orm import declarative_base, sessionmaker

## Promts

All promts be gotten from the [post](https://www.neradot.com/post/building-a-python-react-agent-class-a-step-by-step-guide). Also this promts can/must be rethinked for each task.

In [2]:
REACT_TOOLS_DESCRIPTION = """The available tools are:
{tools_description}

Use the tool name as the "action" value in the JSON response.

"""

REACT_VALID_ACTIONS = """Valid actions are:
- "Final Answer"
- Any of {tools_names}
"""

REACT_JSON_FORMAT = """
Respond in the following $JSON_BLOB format:
<reasoning>
{
        "thought": "" // Explain your reasoning for the chosen action, consider previous and subsequent steps
        "action": "" // The name of the tool to use
        "action_input": {"param_name": param_value} // The input required for the tool
}
</reasoning>
"""

REACT_PROCESS_FORMAT = """
After you select an action, you will receive an observation. Then you can select another action or provide a final answer.
The pattern looks like this:

<reasoning>
$JSON_BLOB
</reasoning>
<observation>
{"observation": // action result}
</observation>
<reasoning>
$JSON_BLOB
</reasoning>
<observation>
{"observation": // action result}
</observation>

... (repeated until you have enough observations to answer the question)

<reasoning>
{
        "thought": // Explain why you have enough information to provide a final answer,
        "action": "Final Answer",
        "action_input": // Your final answer to the question
}
</reasoning>
"""

REACT_ADDITIONAL_INSTRUCTIONS = """
Instructions:
1. Do not use comments in your JSON answer;
2. ALWAYS respond with a valid json blob of a single action;
3. ALWAYS think before choosing an action;
4. Respond in a JSON blob no matter what."""

REACT_INTERMEDIATE_STEPS = """

Here is the user question: 
"{question}"

Here are the intermediate steps so far:
{intermediate_steps}
"""

## Agent

Here present all defenision for the action understanding stage code. This code can be separated and running away of the main service code. Also code can work with different tools for each request. For example user can validate which tool needs for each call. All information about tools stored in the DB (use sqlalchemy driver for working with different db)

### Database with reasone history

In [3]:
Base = declarative_base()


# Agent table store information about all init ReAct actions
# Here we store which question ask user and how many iters we can do
class Agent(Base):
    __tablename__ = "agent"
    id = Column(Integer, primary_key=True)
    uuid = Column(
        String(36), unique=True, index=True, default=lambda: str(uuid.uuid4())
    )
    question = Column(String(1024), nullable=False)
    max_iter = Column(Integer)


# Tool table store information about all tools which can be used in the action
# Here we store all action name and descriptions about the tool according to question
class Tool(Base):
    __tablename__ = "tool"
    id = Column(Integer, primary_key=True)
    agent_id = Column(
        Integer, ForeignKey("agent.id", ondelete="CASCADE"), nullable=False
    )
    name = Column(String(64), nullable=False)
    description = Column(String(1024), nullable=False)


# Step table store information about all intermediate steps during the execution
# Here we store tool id which used in the step and what we send here
class Step(Base):
    __tablename__ = "step"
    id = Column(Integer, primary_key=True)
    agent_id = Column(
        Integer, ForeignKey("agent.id", ondelete="CASCADE"), nullable=False
    )
    thought = Column(String(1024), nullable=False)
    tool_id = Column(Integer, ForeignKey("tool.id", ondelete="CASCADE"), nullable=False)
    tool_input = Column(JSON, nullable=False)


# Result table store information about all result in all intermediate steps during the execution
# Here we store step observation
class Result(Base):
    __tablename__ = "result"
    id = Column(Integer, primary_key=True)
    agent_id = Column(
        Integer, ForeignKey("agent.id", ondelete="CASCADE"), nullable=False
    )
    step_id = Column(Integer, ForeignKey("step.id", ondelete="CASCADE"), nullable=False)
    observation = Column(String(1024), nullable=False)

### Dataclasses for reasone

In [4]:
@dataclass
class ToolData:
    name: str
    description: str


@dataclass
class AgentData:
    id: int
    uuid: str


@dataclass
class ReasonData:
    thought: str
    action: str
    action_input: str
    retry: bool
    is_final: bool


@dataclass
class ResultData:
    observation: str

### ReAct Driver Class

In [5]:
class ReactDriver(object):

    def __init__(self, config: dict) -> None:
        r'''Constructor method.'''
        self.engine = create_async_engine(config["Database"]["url"])
        self.async_session = sessionmaker(self.engine,
                                          class_=AsyncSession,
                                          expire_on_commit=False)

        self.open_ai_connection = config["Model"]["OpenAiConnection"]
        self.open_ai_execution = config["Model"]["OpenAiExecution"]

        self.final_stage_name = "Final Answer"
        self.stop_stage_name = "Stop Max Iter"
        self.error_stage_name = "Error"

    @staticmethod
    def parse_json(string: str) -> str:
        r'''Returns json parsed data from the string.
        
        :param string: The json-like string.
        :type string: str.
        :return: The parsed object from the json-like string.
        :rtype: dict
        '''
        try:
            return json.loads(
                string.replace("<reasoning>", "").replace("</reasoning>",
                                                          "").strip())
        except json.decoder.JSONDecodeError:
            raise ValueError(f"Invalid JSON: {string}")

    async def init_db(self) -> None:
        r'''The database initialisation method.'''
        async with self.engine.begin() as connection:
            await connection.run_sync(Base.metadata.create_all)

    async def create_agent_question(self,
                                    question: str,
                                    tools: list[ToolData] = [],
                                    max_iter: int = 10) -> AgentData:
        r'''Returns created object of question driver.
        
        :param question: The string of question to the model.
        :type question: str.
        :param tools: The list of tools wich model can used during execution.
        :type tools: list[ToolData].
        :param max_iter: The maximum number of iterations for the question execution.
        :type max_iter: int.
        :return: The created object of execution for the question.
        :rtype: AgentData
        '''
        async with self.async_session() as session:
            agent = Agent(question=question, max_iter=max_iter)
            session.add(agent)
            await session.commit()

            for tool in tools:
                session.add(
                    Tool(
                        agent_id=agent.id,
                        name=tool.name,
                        description=tool.description,
                    ),)
            await session.commit()
            agent_uuid = agent.uuid
            agent_id = agent.id

        return AgentData(id=agent_id, uuid=agent_uuid)

    @staticmethod
    async def _get_agent_by_uuid(uuid: str,
                                 session: AsyncSession) -> [int, str, int]:
        r'''Returns information about request by there uuid.
        
        :param uuid: The uuid of the request in the database.
        :type uuid: str.
        :param session: The session to the db.
        :type session: AsyncSession.
        :return: All information about request: (id, question, max_iter).
        :rtype: Tuple[int, str, int]
        '''
        agents = ((await
                   session.execute(select(Agent).where(Agent.uuid == uuid)
                                  )).scalars().all())

        if not agents:
            raise ValueError(f"Bad uuid: {uuid}. Is not presented.")

        for agent in agents:
            return agent.id, agent.question, agent.max_iter

    async def _get_tools_by_agent(self,
                                  agent_id: int,
                                  session: AsyncSession,
                                  extra: bool = False) -> [dict, dict]:
        r'''Returns information about all tools according to the request.
        
        :param agent_id: The id of the request in the database.
        :type agent_id: str.
        :param session: The session to the db.
        :type session: AsyncSession.
        :param extra: The parameter work with need to return extra (not user) tools to the response.
        :type extra: bool.
        :return: All information about tools: (tools, tool_name2tool_id).
        :rtype: Tuple[dict[int, ToolData], dict[str, int]]
        '''
        tools = dict()
        tool_name2tool_id = dict()
        for tool in ((await session.execute(
                select(Tool).where(Tool.agent_id == agent_id)
        )).scalars().all()):
            tools[tool.id] = ToolData(name=tool.name,
                                      description=tool.description)
            tool_name2tool_id[tool.name] = tool.id
        if extra:
            tools[0] = ToolData(name=self.final_stage_name,
                                description=self.final_stage_name)
            tool_name2tool_id[self.final_stage_name] = 0

            tools[-1] = ToolData(name=self.stop_stage_name,
                                 description=self.stop_stage_name)
            tool_name2tool_id[self.stop_stage_name] = -1

            tools[-2] = ToolData(name=self.error_stage_name,
                                 description=self.error_stage_name)
            tool_name2tool_id[self.error_stage_name] = -2
        return tools, tool_name2tool_id

    async def _is_final_max_iter(self, uuid: str) -> bool:
        r'''Check that we are not over max iteration for given request.
        
        :param uuid: The uuid of the request in the database.
        :type uuid: str.
        :return: It's over or can continue driving.
        :rtype: bool
        '''
        async with self.async_session() as session:
            agent_id, question, max_iter = await self._get_agent_by_uuid(
                uuid, session)
            steps = ((await session.execute(
                select(Step).where(Step.agent_id == agent_id))).scalars().all())
            if len(steps) >= max_iter:
                return True
        return False

    async def build_messages(self, uuid: str) -> list[dict]:
        r'''Build messages promt to the LLM according to the request.
        
        :param uuid: The uuid of the request in the database.
        :type uuid: str.
        :return: The messages to the LLM
        :rtype: list[dict[str, str]]
        '''
        async with self.async_session() as session:
            # GET INFORMATION ABOUT CURRENT AGENT REQUEST
            agent_id, question, max_iter = await self._get_agent_by_uuid(
                uuid, session)

            # GET INFORMATION ABOUT ALL TOOLS WITH EXTRA
            tools, _ = await self._get_tools_by_agent(agent_id,
                                                      session,
                                                      extra=True)

            # GET INFORMATION ABOUT ALL PREVIOUS CALLS
            results = dict()
            for result in ((await session.execute(
                    select(Result).where(Result.agent_id == agent_id)
            )).scalars().all()):
                results[result.step_id] = result.observation

            steps = ((await session.execute(
                select(Step).where(Step.agent_id == agent_id))).scalars().all())
            intermediate_steps = []
            for step in steps:
                intermediate_steps.append({
                    "id": step.id,
                    "thought": step.thought,
                    "action": tools[step.tool_id].name,
                    "action_input": json.loads(step.tool_input),
                    "observation": results.get(step.id, ""),
                })

            intermediate_steps = json.dumps(intermediate_steps)

            # GET INFORMATION ABOUT ALL TOOLS FOR GET ACTIONS
            tools, _ = await self._get_tools_by_agent(agent_id, session)

            tools_description = "\n".join([
                f"{tool.name}: {tool.description.strip()}"
                for tool in tools.values()
            ])
            tools_names = [tool.name for tool in tools.values()]

        system_prompt_message = (
            REACT_TOOLS_DESCRIPTION.format(tools_description=tools_description)
            + REACT_VALID_ACTIONS.format(tools_names=tools_names) +
            REACT_JSON_FORMAT + REACT_PROCESS_FORMAT +
            REACT_ADDITIONAL_INSTRUCTIONS + REACT_INTERMEDIATE_STEPS.format(
                question=question, intermediate_steps=intermediate_steps))

        messages = [{"role": "system", "content": system_prompt_message}]
        return messages

    async def reason(self, uuid: str) -> ReasonData:
        r'''Generate an action for given state.
        
        :param uuid: The uuid of the request in the database.
        :type uuid: str.
        :return: The information about current reasone step. What we can do next.
        :rtype: ReasonData
        '''
        if await self._is_final_max_iter(uuid):
            return ReasonData(
                thought="It's over by maximum iterations.",
                action=self.stop_stage_name,
                action_input="",
                retry=False,
                is_final=True,
            )

        completion = ((await AsyncOpenAI(
            **self.open_ai_connection).chat.completions.create(
                messages=await self.build_messages(uuid),
                **self.open_ai_execution)).choices[0].message.content)

        try:
            parsed_completion = self.parse_json(completion)

            return ReasonData(
                thought=parsed_completion["thought"],
                action=parsed_completion["action"],
                action_input=parsed_completion["action_input"],
                retry=False,
                is_final=parsed_completion["action"] == self.final_stage_name,
            )
        except Exception:
            return ReasonData(
                thought="Something went wrong... Try new way...",
                action=self.error_stage_name,
                action_input=traceback.format_exc(),
                retry=True,
                is_final=False,
            )

    async def realize(self, uuid: str, reason: ReasonData,
                      result: ResultData) -> None:
        r'''Generate an action for given state.
        
        :param uuid: The uuid of the request in the database.
        :type uuid: str.
        :param reason: The information about current reasone step. What we have already done.
        :type reason: ReasonData.
        :param result: The result of our reasone execution.
        :type result: ResultData.
        '''
        async with self.async_session() as session:
            agent_id, question, max_iter = await self._get_agent_by_uuid(
                uuid, session)
            tools, tool_name2tool_id = await self._get_tools_by_agent(
                agent_id, session, extra=True)

            step = Step(
                agent_id=agent_id,
                thought=reason.thought,
                tool_id=tool_name2tool_id[reason.action],
                tool_input=json.dumps(reason.action_input),
            )
            session.add(step)
            await session.commit()

            result = Result(agent_id=agent_id,
                            step_id=step.id,
                            observation=result.observation)
            session.add(result)
            await session.commit()

### ReAct initialisation

In [6]:
config = {
    "Database": {
        "url": "sqlite+aiosqlite:///:memory:"
    },
    "Model": {
        "OpenAiConnection": {
            "api_key": "<your secret api key>",
        },
        "OpenAiExecution": {
            "model": "gpt-4.1-mini",
            "temperature": 0,
            "stop": ["</reasoning>"],
        },
    },
}

driver = ReactDriver(config)
await driver.init_db()


## External Code which use our ReAct

### Actions

In [7]:
def calculator(expression: str) -> float:
    r'''Generate an action for given state.

    :param expression: The string of expression
    :type expression: str.
    :return: The result of the expression
    :rtype: float
    
    Example:
    
    >>> calculator(expression="2+2*2")
    6
    '''
    return eval(expression)


def echo(x: str, n: int) -> str:
    r'''Repeat the input string x, n times.

    :param x: The string to repeat
    :type x: str.
    :param n: Number of times to repeat the string
    :type n: int.
    :return: The input string repeated n times
    :rtype: str
    
    Example:
    
    >>> echo(x="a", n=10)
    "aaaaaaaaaa"
    '''
    return x * n

### Action driver

In [9]:
tools_dict = {tool.__name__: tool for tool in [calculator, echo]}

agent_data = await driver.create_agent_question(
    "Расчитай следующее выражение: 2**10. После чего посчитай корень квадратный от этого числа. Ответом должно быть только число.",
    [
        ToolData(name=name, description=tools_dict[name].__doc__)
        for name in tools_dict
    ],
    max_iter=10,
)


is_final = False

while not is_final:
    reason = await driver.reason(agent_data.uuid)
    is_final = reason.is_final

    if not reason.is_final and not reason.retry:
        observation = None
        try:
            if isinstance(reason.action_input, dict):
                observation = tools_dict[reason.action](**reason.action_input)
            else:
                observation = tools_dict[reason.action](reason.action_input)
        except Exception:
            observation = traceback.format_exc()
        
        result = ResultData(
            observation=observation
        )
    else:
        result = ResultData(observation=reason.action_input)

    await driver.realize(agent_data.uuid, reason, result)

    print("-" * 80)
    print(reason)
    print(result)

--------------------------------------------------------------------------------
ReasonData(thought='First, I need to calculate 2 raised to the power of 10 (2**10). After that, I will calculate the square root of the resulting number. I will start by calculating 2**10.', action='calculator', action_input={'expression': '2**10'}, retry=False, is_final=False)
ResultData(observation=1024)
--------------------------------------------------------------------------------
ReasonData(thought='I have calculated 2**10 and the result is 1024. Now I need to calculate the square root of 1024.', action='calculator', action_input={'expression': 'sqrt(1024)'}, retry=False, is_final=False)
ResultData(observation='Traceback (most recent call last):\n  File "/tmp/ipykernel_859109/213637387.py", line 23, in <module>\n    observation = tools_dict[reason.action](**reason.action_input)\n  File "/tmp/ipykernel_859109/3839731923.py", line 14, in calculator\n    return eval(expression)\n  File "<string>", line 